# TALUS Demo
Tag And Location Unification System

The goal of this demo is to satisfy the 'Preprocessing' step detailed in [our outline](https://beanland.notion.site/TALUS-d223e68cb6a5480699ceb53530398af8)

I will create a function that will: take parameters to create an address and tag object with the [Samsara API](https://developers.samsara.com/docs); and add a tag to each Samsara asset that exists within its bounding box. 
``` typescript
    function preprocess(addressName: string, tagName: string, geofence: List[Tuple[number, number]]) -> void
```

Specifically, I will use CES Memphis, but initially I will create a test address called "Test CES Memphis", which I will delete later. 

## Step 1: Exploration
I want to test that I can filter Assets by address, and verify how many devices are within CES Memphis

In [8]:
const sdk = require('api')('@samsara-dev-rel/v2019.01.01#2xqng621f6l8vvz9pu');

In [ ]:
require('dotenv').config({path:__dirname+'./.env'})
var STOKEN = process.env.SAMSARA_TOKEN

In [5]:
var assets;

sdk.auth(STOKEN);
sdk.V1getAllAssetCurrentLocations()
  .then(res => {
    console.log(res, "Length", res["assets"].length)
    assets = res["assets"]
  })
  .catch(err => console.error(err));

Promise { <pending> }

{
  assets: [
    {
      id: 281474979023263,
      name: '150KIEU15',
      assetSerialNumber: '150KIEU15',
      engineHours: 0,
      cable: [Object],
      location: [Array]
    },
    {
      id: 281474979023264,
      name: '300K3CO79',
      assetSerialNumber: '300K3CO79',
      engineHours: 0,
      cable: [Object],
      location: [Array]
    },
    {
      id: 281474979023266,
      name: '150K3CO82',
      assetSerialNumber: '150K3CO82',
      engineHours: 0,
      cable: [Object],
      location: [Array]
    },
    {
      id: 281474979023267,
      name: '300K3CO08',
      assetSerialNumber: '300K3CO08',
      engineHours: 0,
      cable: [Object],
      location: [Array]
    },
    {
      id: 281474979023271,
      name: '75K3C025',
      assetSerialNumber: '75K3C025',
      engineHours: 0,
      cable: [Object],
      location: [Array]
    },
    {
      id: 281474979157819,
      name: 'Football 2',
      assetSerialNumber: 'G2CY-Y8Z-4XF',
      engineHours: 0,
      

In [6]:
const MEMPHIS = "CES Memphis"

In [10]:
var memphis_assets = assets.filter(e => e["location"][0]["location"] === MEMPHIS)
console.log("There are", memphis_assets.length, "assets in", MEMPHIS)

There are 355 assets in CES Memphis


Checking on the Samsara [cloud portal](https://cloud.samsara.com/o/10001042/fleet/list?bounds=28.564133%2C-101.170133%2C44.581756%2C-75.659879&z=6), we can see visually that there are around this number of assets in the Memphis area. Positive discrepancies can be accounted for with devices in the greater Memphis area. 
![Cloud image](./img/cloud_image.png)

## Step 2: Creating Objects
Next, I want to create a new Address object, then a new tag object with the API. These are both simple API calls, however I need a Geofence Object, which I will get from the existing CES Memphis Address Object

### Geofence Object
In the API, there are 2 options for creating a Geofence:
* Circle
* Polygon

In the existing Memphis site, it is defined as a circle, but in practice we will use the polygon stored about the Skyfall map. A polygon requires at least 3 but no more than 40 vertices. The Skyfall DB does not expressly store the geometry of the map, but does store the centre (lat, lng) and the image. From those two, I can try to figure out the bounds, but that is down the road

For this demo, I will be using the Circle option, as that is what already exists, but this can easily be changed for practice. 

In [5]:
const MEMPHIS_ID = 26709829 // retreived from API
const GEOFENCE = {"circle": {
    "latitude": 35.05687169999999,
    "longitude": -89.9464868,
    "radiusMeters": 2555
  }} 
const TEST_ADDRESS = "Test CES Memphis"
const TEST_TAG = "Test_0001"

### Create Address

Addresses to delete: 40108928,40109193,40110375, '40110763', '40110974'

In [51]:
var params = {
    geofence: GEOFENCE, 
    name: TEST_ADDRESS,
    formattedAddress: "3500 Air Center Cove, Memphis, TN 38118, USA"
}

sdk.auth(STOKEN);
sdk.createAddress(params)
  .then(res => console.log(res))
  .catch(err => console.error(err));

Promise { <pending> }

{
  data: {
    id: '40110375',
    name: 'Test CES Memphis',
    createdAtTime: '2022-10-06T15:12:30.782034755Z',
    formattedAddress: '3500 Air Center Cove, Memphis, TN 38118, USA',
    geofence: { circle: [Object] },
    latitude: 35.05687169999999,
    longitude: -89.9464868
  }
}


In [ ]:
function createAddress(name, geofence, formattedAddress)

### Create Tag
Tags to delete: 3759296

In [15]:
tagParams = {name: TEST_TAG}

sdk.auth(STOKEN);
sdk.createTag(tagParams)
  .then(res => console.log(res))
  .catch(err => console.error(err));

Promise { <pending> }

{
  data: {
    id: '3759296',
    name: 'Test_0001',
    vehicles: [],
    drivers: [],
    assets: [],
    machines: [],
    sensors: [],
    addresses: [],
    externalIds: { 'samsara.name': 'Test_0001' }
  }
}


In [18]:
var tag_id = 3759296

## Step 3: Adding Tags to Assets
* Filter assets by location
* Add tags to those assets
* Bish Bash Bosh

In [42]:
async function createTagWithAssets(addressName, tagName) {
    return new Promise(async (resolve, reject) => {
        sdk.auth(STOKEN);
        var assets = (await sdk.V1getAllAssetCurrentLocations())["assets"]

        var filtered_assets = assets.filter(e => e["location"][0]["location"] === addressName)
        var assetIDs = filtered_assets.map(e => e["id"])
        sdk.createTag({name: tagName, assets: assetIDs}).then(res => {
            resolve(res["data"]["id"])
        })
    })
}

var resultID; 
createTagWithAssets(TEST_ADDRESS, "Test_0002")
    .then(id => {
        resultID = id
        console.log("Success! New ID: ", id)
    })
    .catch(err => console.error(err))

Promise { <pending> }

Success! New ID:  3759320


### Testing
Success criteria
* 355 assets in tag

In [45]:
const NUM_MEMPHIS_ASSETS = 355

In [50]:
sdk.getTag({id: resultID})
  .then(res => {
    console.log(res)
    if (res["data"]["assets"].length != NUM_MEMPHIS_ASSETS)
      throw Error("Inocorrect number of assets: expected: ", NUM_MEMPHIS_ASSETS, "Found:", res["data"]["assets"].length)
    else
      console.log("Success!")
    })
.catch(err => console.error(err));

Promise { <pending> }

{
  data: {
    id: '3759320',
    name: 'Test_0002',
    vehicles: [],
    drivers: [],
    assets: [],
    machines: [],
    sensors: [],
    addresses: [],
    externalIds: { 'samsara.name': 'Test_0002' }
  }
}


Error: Inocorrect number of assets: expected: 
    at evalmachine.<anonymous>:5:13
    at runMicrotasks (<anonymous>)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
